In [ ]:
from langchain_docling import DoclingLoader

In [ ]:
sourece = "ReAct.pdf"
loader = DoclingLoader(
    file_path=sourece,
    export_type = "markdown"
)
documents = loader.load()

In [ ]:
print(documents[0].page_content[:500])  # Print the first 500 characters of the first document

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Docling 마크다운 출력용 섹션 단위 분할
text_splitter = RecursiveCharacterTextSplitter(
    # 청크 최대 크기: 1200자
    # - 영어 논문 1200자 ≈ 300토큰 (대부분 임베딩 모델 512토큰 제한 내)
    # - 논문 섹션(Abstract, Introduction) 전체를 한 청크로 유지 가능
    # - 너무 작으면 문맥 손실, 너무 크면 검색 정확도 하락
    chunk_size=1200,
    
    # 청크 간 겹치는 부분: 200자 (chunk_size의 16%)
    # - 섹션 경계에서 문맥이 끊기는 것 방지
    # - 검색 시 경계 부근 정보 놓치지 않도록
    # - 일반적으로 chunk_size의 10-20% 권장
    chunk_overlap=200,
    
    # 분할 우선순위 (위에서부터 순서대로 시도)
    separators=[
        "\n## ",      # 1순위: 논문 섹션 헤더 (## Abstract, ## Introduction)
                      # Docling이 주요 섹션을 2레벨 헤더로 변환하므로
                      # 가장 큰 의미 단위인 섹션을 먼저 보존
        
        "\n### ",     # 2순위: 서브섹션 (### 3.1 Dataset, ### 3.2 Model)
                      # 섹션이 1200자 초과 시 서브섹션 단위로 자연스럽게 분할
        
        "\n\n",       # 3순위: 문단 경계
                      # 서브섹션도 길면 문단 단위로 분할
                      # 논문은 문단이 논리적 단위이므로 여기서 자르는 게 자연스러움
        
        "\n",         # 4순위: 줄바꿈
                      # 문단도 길면 줄 단위로
                      # 수식, 리스트, 코드 블록 등은 줄바꿈으로 구분됨
        
        ". ",         # 5순위: 문장 끝 (마침표 + 공백)
                      # 최소한 완전한 문장은 유지
                      # 공백까지 포함해야 문장 끝 정확히 감지
        
        " ",          # 6순위: 단어 경계 (공백)
                      # 문장도 초과하면 단어 단위로
                      # 최소한 단어는 쪼개지지 않게
        
        ""            # 7순위: 강제 분할 (문자 단위)
                      # 긴 URL, 코드, 수식 등 chunk_size 초과 시 문자로 강제 분할
                      # 에러 방지용 최후의 안전장치
    ],
    
    # 원본 문서에서의 시작 위치를 메타데이터에 추가
    # - RAG 검색 후 출처 추적 가능 (몇 번째 문자/페이지에서 왔는지)
    # - chunk.metadata['start_index']로 접근
    # - 사용자에게 "논문 3페이지 Introduction 섹션" 같은 정보 제공 가능
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)

In [ ]:
print(chunks[2])

In [ ]:
from FlagEmbedding import BGEM3FlagModel

In [ ]:
embedding_model = BGEM3FlagModel("BAAI/bge-m3")
embedding_model.model.to("cuda")
embedding_model.model.half()

In [ ]:
print(next(embedding_model.model.parameters()).device)
print(next(embedding_model.model.parameters()).dtype)

In [ ]:
texts = [chunk.page_content for chunk in chunks]

In [ ]:
print(texts)

In [ ]:
len(texts)

In [ ]:
outputs = embedding_model.encode(
    texts,
    batch_size = 12,
    max_length = 8192,
    return_dense = True,
    return_sparse = True,
    return_colbert_vecs=False,
)

In [ ]:
outputs['dense_vecs'][0][:5]

In [ ]:
list(outputs['lexical_weights'][0].items())[:5] 

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
client = OpenSearch(
    hosts = [
        {
            "host": "https://search-papergraph-fonrzpiqlo6hbcwkufdwuukd6y.us-east-1.es.amazonaws.com",
            "port": 443
        }
        ],
    http_auth = ('wnghks5432', 'Wnghks1278@'),
    use_ssl = True,
    verify_certs = False
)

In [ ]:
index_name = "papers"

index_body = {
    "settings":{
        "index":{
            "knn": True,
            "knn.algo_param.ef_search":100
        }
    },
    "mappings":{
        "properties":{
            "text":{
                "type":"text",
                "analyzer":"standard"
            },
            "metadata":{
                "type":"object",
                "enabled": True
            },
            "dense_vector":{
                "type":"knn_vector",
                "dimension": 1024,
                "method":{
                    "name":"hnsw",
                    "space_type":"cosinesimil",
                    "engine":"faiss",
                    "parameters":{
                        "ef_construction":128,
                        "m":16
                    }
                }
            },
            "sparse_vector":{
                "type":"rank_features",
            }
        }
    }
}

In [ ]:
if client.indices.exists(index_name):
    client.indices.delete(index=index_name)
    
response = client.indices.create(index=index_name, body=index_body)

In [ ]:
from tqdm import tqdm

# bulk 색인용 액션 리스트 생성
# RAG 시스템을 처음 구축하거나 대량의 문서를 벡터화해서 넣을 때
actions = []
for i,chunk in tqdm(enumerate(chunks,desc="Indexing documents")):
    action = {
        "_index": index_name,
        "_id": i,
        "_body":{
            "text": chunk.page_content,
            "metadata": chunk.metadata,
            "dense_vector": outputs['dense_vecs'][i].tolist(),
            "sparse_vector": outputs['sparse_vecs'][i],
        }
    }
    actions.append(action)



In [ ]:
success,failed = client.bulk(
    client,
    actions,
    chunk_size=500,
    request_timeout=60
)
print(f"Successfully indexed {success} documents, failed to index {failed} documents.")

In [ ]:
# 개별 문서 색인
# 사용자가 게시글 하나를 작성해서 즉시 검색 엔진에 반영해야 할 때
for i,chunk in enumerate(chunks):
    doc = {
        "text": chunk.page_content,
        "metadata": chunk.metadata,
        "dense_vector": outputs['dense_vecs'][i].tolist(),
        "sparse_vector": outputs['sparse_vecs'][i],
    }
    response = client.index(
        index=index_name,
        id=i,
        body=doc,
        refresh=False
    )
    
client.indices.refresh(index=index_name)


In [ ]:
# ============================================
# Search Pipeline 생성: Hybrid Search 결과 정규화 및 병합
# ============================================
client.http.put(
    # Pipeline 이름 지정 (사용자 정의 가능)
    # 나중에 검색 시 params={"search_pipeline": "이 이름"} 으로 호출
    "/_search/pipeline/hybrid_norm_rrf",
    
    body={
        # response_processors: 검색 결과를 어떻게 처리할지 정의
        # (검색 쿼리 실행 → 결과 반환 전에 이 프로세서들이 순차 실행됨)
        "response_processors": [
            {
                # normalization-processor: 점수 정규화 및 병합을 담당하는 프로세서
                "normalization-processor": {
                    
                    # ===== Step 1: 정규화 (Normalization) =====
                    # Dense 검색과 Sparse 검색의 점수 스케일이 다르므로
                    # 0~1 사이로 정규화해서 공정하게 비교 가능하도록 함
                    "normalization": {
                        # technique: 정규화 방법 선택
                        # - "min_max": (score - min) / (max - min) 방식
                        # - "l2": L2 norm 방식 (벡터 크기로 나눔)
                        # - "z_score": 평균/표준편차 기반 정규화
                        "technique": "min_max"  # 가장 일반적이고 직관적
                    },
                    
                    # ===== Step 2: 병합 (Combination) =====
                    # 정규화된 Dense/Sparse 결과를 하나로 합치는 방법
                    "combination": {
                        # technique: 병합 알고리즘 선택
                        # - "rrf": Reciprocal Rank Fusion (순위 기반 병합)
                        # - "arithmetic_mean": 산술 평균 (점수 평균)
                        # - "geometric_mean": 기하 평균
                        # - "harmonic_mean": 조화 평균
                        "technique": "rrf",  # 검색에서 가장 많이 쓰임
                        
                        # parameters: RRF 알고리즘의 하이퍼파라미터
                        "parameters": {
                            # rank_constant (k): RRF 공식의 상수
                            # 
                            # RRF 공식: score = 1 / (k + rank)
                            # 
                            # 예시:
                            # - Dense 검색 1위 문서: 1/(60+1) = 0.0164
                            # - Sparse 검색 3위 문서: 1/(60+3) = 0.0159
                            # → 두 점수를 합산해서 최종 순위 결정
                            #
                            # k 값의 의미:
                            # - 작을수록 (예: 10): 상위 랭크 차이를 크게 봄
                            # - 클수록 (예: 100): 랭크 차이를 완만하게 봄
                            # - 기본값 60: 경험적으로 가장 안정적
                            "rank_constant": 60
                        }
                    }
                }
            }
        ]
    }
)

# ============================================
# 동작 흐름 요약
# ============================================
# 1. 사용자가 hybrid 쿼리 전송 (Dense + Sparse)
#    → OpenSearch가 각각 검색 수행
#
# 2. Dense 결과: [문서A: 0.95, 문서B: 0.82, 문서C: 0.71]
#    Sparse 결과: [문서C: 8.2, 문서A: 5.1, 문서D: 3.9]
#    → 점수 스케일이 다름! (0~1 vs 1~10)
#
# 3. Min-Max 정규화 실행:
#    Dense → [문서A: 1.0, 문서B: 0.46, 문서C: 0.0]
#    Sparse → [문서C: 1.0, 문서A: 0.28, 문서D: 0.0]
#
# 4. RRF 병합 (각 문서의 순위 기반):
#    문서A: 1/(60+1) + 1/(60+2) ≈ 0.0325
#    문서C: 1/(60+3) + 1/(60+1) ≈ 0.0323
#    문서B: 1/(60+2) + 0 ≈ 0.0161
#    문서D: 0 + 1/(60+3) ≈ 0.0159
#
# 5. 최종 결과: [문서A, 문서C, 문서B, 문서D] 순으로 반환
# ============================================

In [ ]:
# ============================================
# Search Pipeline 조회: 생성된 Pipeline 설정 확인
# ============================================

# HTTP GET 요청으로 특정 Pipeline의 설정을 가져옴
pipeline = client.http.get(
    # 조회할 Pipeline 이름 지정
    # (앞에서 생성한 "hybrid_norm_rrf" Pipeline의 설정을 확인)
    "/_search/pipeline/hybrid_norm_rrf"
)

# 반환값 출력
# - Pipeline이 존재하면: 설정 내용 (dict 형태)
# - Pipeline이 없으면: 404 에러 발생
print(pipeline)

# ============================================
# 반환 예시
# ============================================
# {
#     "hybrid_norm_rrf": {
#         "response_processors": [
#             {
#                 "normalization-processor": {
#                     "normalization": {"technique": "min_max"},
#                     "combination": {
#                         "technique": "rrf",
#                         "parameters": {"rank_constant": 60}
#                     }
#                 }
#             }
#         ]
#     }
# }

In [ ]:
def search_papers(query, top_k=5):
    """
    Hybrid Search 함수: Dense(의미 검색) + Sparse(키워드 검색) 결합
    
    동작 원리:
    1. 쿼리를 Dense 벡터 + Sparse 가중치로 인코딩
    2. OpenSearch에 hybrid 쿼리 전송 (Dense kNN + Sparse rank_feature)
    3. Search Pipeline(hybrid_norm_rrf)이 결과를 정규화 및 병합
    4. 최종 통합 결과를 점수 순으로 반환

    Args:
        query (str): 검색 쿼리 (예: "attention mechanism")
        top_k (int): 반환할 문서 개수 (기본 5개)

    Returns:
        list[dict]: 검색 결과 리스트 (점수 내림차순)
            - text: 청크 내용
            - metadata: 메타데이터 (파일명, 페이지 등)
            - score: 검색 점수 (높을수록 관련성 높음, RRF로 계산됨)
    """

    # ============================================
    # 1. 쿼리 인코딩 (BGE-M3 모델 사용)
    # ============================================
    # BGE-M3는 하나의 모델로 Dense와 Sparse 임베딩을 동시 생성
    query_output = embedding_model.encode(
        [query],  # 배치 형태로 입력 (리스트로 감싸기)
        return_dense=True,   # Dense 벡터 반환 (1024차원 벡터)
        return_sparse=True   # Sparse 가중치 반환 (단어:가중치 딕셔너리)
    )
    # 반환값 예시:
    # {
    #     'dense_vecs': [[0.123, -0.456, ...]],  # shape: (1, 1024)
    #     'lexical_weights': [{'attention': 0.8, 'mechanism': 0.6, ...}]
    # }

    # ============================================
    # 2. Hybrid Search 쿼리 구성
    # ============================================
    search_body = {
        # 반환할 문서 개수 제한
        "size": top_k,

        # 응답에 포함할 필드 지정 (벡터는 용량이 크므로 제외)
        # dense_vector(1024차원), sparse_vector(수천 단어) 필드는 반환 안 함
        "_source": ["text", "metadata"],

        "query": {
            # hybrid: OpenSearch의 하이브리드 검색 쿼리
            # (여러 쿼리를 동시 실행하고 Search Pipeline이 결과 병합)
            "hybrid": {
                "queries": [
                    # ===== 쿼리 1: Dense 검색 (의미 기반) =====
                    {
                        "knn": {  # k-Nearest Neighbors (벡터 유사도 검색)
                            "dense_vector": {  # 검색할 벡터 필드명
                                # 쿼리 벡터 (1024차원 리스트로 변환)
                                "vector": query_output['dense_vecs'][0].tolist(),

                                # 찾을 이웃 개수
                                # (HNSW 그래프에서 top_k개 이웃 탐색)
                                "k": top_k
                            }
                        }
                    },

                    # ===== 쿼리 2: Sparse 검색 (키워드 기반) =====
                    {
                        "bool": {  # 불리언 쿼리 (여러 조건 조합)
                            "should": [  # OR 조건 (하나라도 매칭되면 점수 부여)
                                {
                                    "rank_feature": {  # 순위 특성 쿼리 (0~1 정규화된 값)
                                        # 필드명: sparse_vector.단어
                                        # (동적 매핑으로 각 단어가 별도 필드로 저장됨)
                                        "field": f"sparse_vector.{word}",

                                        # 쿼리에서의 해당 단어 중요도
                                        # (BGE-M3가 계산한 가중치 사용)
                                        "boost": score
                                    }
                                }
                                # 쿼리의 모든 중요 단어에 대해 반복
                                # 예: {'attention': 0.8, 'mechanism': 0.6}
                                # → 2개의 rank_feature 쿼리 생성
                                for word, score in query_output['lexical_weights'][0].items()
                            ]
                        }
                    }
                ]
            }
        }
    }
    # ============================================
    # 쿼리 실행 흐름:
    # 1. OpenSearch가 Dense와 Sparse 쿼리를 병렬 실행
    # 2. 각각 독립적으로 top_k개 문서 검색
    # 3. Search Pipeline(hybrid_norm_rrf)이 두 결과를 받아서:
    #    - Min-Max 정규화 (점수 스케일 통일)
    #    - RRF 알고리즘으로 병합 (순위 기반)
    # 4. 최종 통합 결과 반환
    # ============================================

    # ============================================
    # 3. 검색 실행
    # ============================================
    response = client.search(
        # 검색할 인덱스명
        index=index_name,
        
        # 쿼리 본문 (위에서 구성한 hybrid 쿼리)
        body=search_body,
        
        # Search Pipeline 지정
        # - "hybrid_norm_rrf" Pipeline이 응답 처리
        # - 정규화(min_max) + 병합(rrf) 자동 수행
        # - 이 파라미터 없으면 원시 점수 그대로 반환됨
        params={"search_pipeline": "hybrid_norm_rrf"}
    )
    # ============================================
    # response 구조 예시:
    # {
    #     "hits": {
    #         "total": {"value": 100},
    #         "max_score": 0.0325,
    #         "hits": [
    #             {
    #                 "_score": 0.0325,  # RRF로 계산된 최종 점수
    #                 "_source": {
    #                     "text": "Attention mechanism은...",
    #                     "metadata": {"file": "paper.pdf", "page": 3}
    #                 }
    #             },
    #             ...
    #         ]
    #     }
    # }
    # ============================================

    # ============================================
    # 4. 결과 파싱 (사용자 친화적 형태로 변환)
    # ============================================
    results = []
    for hit in response["hits"]["hits"]:
        results.append({
            # 문서 텍스트 (청크 내용)
            "text": hit["_source"]["text"],
            
            # 메타데이터 (출처 정보)
            "metadata": hit["_source"]["metadata"],
            
            # 최종 검색 점수 (RRF로 계산됨)
            # - 범위: 보통 0.001 ~ 0.05 사이
            # - 높을수록 쿼리와 관련성 높음
            "score": hit["_score"]
        })

    # 결과 리스트 반환 (이미 점수 내림차순 정렬됨)
    return results


# ============================================
# 사용 예시
# ============================================
# results = search_papers("attention mechanism", top_k=5)
# 
# for i, result in enumerate(results, 1):
#     print(f"{i}. Score: {result['score']:.4f}")
#     print(f"   Text: {result['text'][:100]}...")
#     print(f"   Source: {result['metadata']['file']}")
#     print()

In [ ]:
# 검색 예시
query = "transformer attention mechanism"
results = search_papers(query, top_k=3)

print(f"\n🔍 검색어: '{query}'")
print(f"📊 결과: {len(results)}개\n")

for i, result in enumerate(results, 1):
    print(f"{'='*80}")
    print(f"📄 결과 {i} | 점수: {result['score']:.4f}")
    print(f"{'='*80}")

    # 텍스트 (처음 300자만)
    text = result['text']
    print(f"텍스트:\n{text[:300]}...")

    # 메타데이터
    print(f"\n메타데이터:")
    for key, value in result['metadata'].items():
        print(f"  - {key}: {value}")

    print()
